In [1]:
# For this visualization project we will use pandas and altair
import pandas as pd
import altair as alt

In [2]:
# The data is provided by the California Water Resources Board
data_path = "https://data.cnra.ca.gov/dataset/6d8745c3-b781-4b95-93b6-a8b5b6f70cd5/resource/afdfdc5b-ad81-4d98-bcd1-e65200ef36d6/download/reservoir_elevation_storage_ab1755.csv"
raw_reservoir_data = pd.read_csv(data_path)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


The cells below encompass the data cleaning process.

In [3]:
raw_reservoir_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276395 entries, 0 to 276394
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   date            276395 non-null  object
 1   field_division  276395 non-null  object
 2   reservoir       276395 non-null  object
 3   elevation       276330 non-null  object
 4   storage         276377 non-null  object
dtypes: object(5)
memory usage: 10.5+ MB


In [4]:
raw_reservoir_data.describe()

,date,field_division,reservoir,elevation,storage
count,276395,276395,276395,276330,276377
unique,15646,5,20,52059,53459
top,2017-01-15,Oroville,OROVILLE,602.60,826
freq,19,92408,15645,1115,1115


In [5]:
raw_reservoir_data.isnull().any()

date              False
field_division    False
reservoir         False
elevation          True
storage            True
dtype: bool

In [6]:
raw_reservoir_data.dropna(axis=0, inplace=True)

In [7]:
raw_reservoir_data["date"] = pd.to_datetime(raw_reservoir_data["date"], errors = "coerce")
recast_reservoir_data = raw_reservoir_data.convert_dtypes()
recast_reservoir_data["elevation"] = pd.to_numeric(recast_reservoir_data["elevation"], errors = "coerce")
recast_reservoir_data["storage"] = pd.to_numeric(recast_reservoir_data["storage"], errors = "coerce")

In [8]:
recast_reservoir_data.dropna(axis=0, inplace=True)
recast_reservoir_data.columns = ["Date", "Field Division", "Reservoir", "Elevation", "Storage"]
recast_reservoir_data.info()
recast_reservoir_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276301 entries, 0 to 276394
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            276301 non-null  datetime64[ns]
 1   Field Division  276301 non-null  string        
 2   Reservoir       276301 non-null  string        
 3   Elevation       276301 non-null  float64       
 4   Storage         276301 non-null  float64       
dtypes: datetime64[ns](1), float64(2), string(2)
memory usage: 12.6 MB


,Date,Field Division,Reservoir,Elevation,Storage
0,1981-01-01,Delta,Clifton Ct,0.30,18909.0
1,1981-01-02,Delta,Clifton Ct,0.30,18909.0
2,1981-01-03,Delta,Clifton Ct,0.34,18995.0
3,1981-01-04,Delta,Clifton Ct,0.32,18952.0
4,1981-01-05,Delta,Clifton Ct,0.32,18952.0


In [9]:
recast_reservoir_data["Year"] = recast_reservoir_data["Date"].dt.year
recast_reservoir_data["Day_of_Year"] = recast_reservoir_data["Date"].dt.dayofyear
recast_reservoir_data["Month"] = recast_reservoir_data["Date"].dt.month

For this assesment we will only be considering the top 3 reservoirs in terms of storage (measured in acre feet) because altair runs slowly for data sets larger than 5000 units.

In [10]:
Top_3 = ["OROVILLE", "San Luis", "CASTAIC"]
Top_3_Data = recast_reservoir_data[recast_reservoir_data["Reservoir"].isin(Top_3)]
Top_3_Data = Top_3_Data[Top_3_Data["Year"] >= 1971]

In [11]:
Maximum_capacity_dict = {'OROVILLE' : 3537577, 'CASTAIC' : 323700, 'San Luis' : 2041000}

In [12]:
Top_3_Filled_Capacity = Top_3_Data.copy()
Top_3_Filled_Capacity["Capacity"] = Top_3_Filled_Capacity["Reservoir"].map(Maximum_capacity_dict)
Top_3_Filled_Capacity["Percent Filled"] = 100 * Top_3_Filled_Capacity["Storage"] / Top_3_Filled_Capacity["Capacity"]
Top_3_Filled_Capacity_Monthly_Average = Top_3_Filled_Capacity.groupby(["Reservoir", "Year", "Month"], as_index=False)["Percent Filled"].mean()
Top_3_Filled_Capacity_Monthly_Average = pd.DataFrame(Top_3_Filled_Capacity_Monthly_Average)
Top_3_Filled_Capacity_Monthly_Capacity_Average = Top_3_Filled_Capacity.groupby(["Reservoir", "Year", "Month"], as_index=False)["Storage"].mean()
Top_3_Filled_Capacity_Monthly_Average = pd.merge(Top_3_Filled_Capacity_Monthly_Average, Top_3_Filled_Capacity_Monthly_Capacity_Average, on=["Reservoir", "Month", "Year"], how = "left")

In [13]:
Overall_Average_Monthly_Storage = Top_3_Filled_Capacity_Monthly_Average.groupby(["Year"], as_index = False).mean()
Overall_Average_Monthly_Storage = pd.DataFrame(Overall_Average_Monthly_Storage)

In [14]:
brush = alt.selection_multi(encodings = ['x'], fields = ['Year'])
color = alt.condition(brush,
                      alt.Color('Year:O', legend = None),
                      alt.value('lightgray'))
time_selection = alt.Chart(Overall_Average_Monthly_Storage).mark_bar().encode(
    x = "Year:O",
    y = "Storage",
    color = color
).add_selection(
    brush
)
time_selection

alt.Chart(...)

In [15]:
Start_Value = Top_3_Filled_Capacity_Monthly_Average.groupby(["Reservoir", "Year"], as_index = False)["Percent Filled"].first()
Start_Value.columns = ["Reservoir", "Year", "Starting Percent Filled"]
Top_3_Filled_Capacity_Monthly_Average = pd.merge(Start_Value, Top_3_Filled_Capacity_Monthly_Average, how = "right", on = ["Reservoir", "Year"])
Top_3_Filled_Capacity_Monthly_Average["Delta Percent Filled"] = Top_3_Filled_Capacity_Monthly_Average["Percent Filled"] - Top_3_Filled_Capacity_Monthly_Average["Starting Percent Filled"]

In [16]:
#Slider Selector
#slider = alt.binding_range(min = 1999, max = 2021, step = 1, name = 'Year Range')
#selector = alt.selection_single(name = 'SelectorName', fields = ['Year Range'], bind = slider, init = {'Year Range': 1999})
#color = alt.condition(
#    alt.datum.xval < selector.cutoff,
#    alt.value('red'), alt.value('blue')
#)


#Legend Selector
selector = alt.selection_multi(fields = ['Year'])
color = alt.condition(selector,
                     alt.Color('Year:O', legend = None),
                     alt.value('lightgray'))


#Dropdown
#input_dropdown = alt.binding_select(options = [x for x in range(1999,2022,1)])
#selector = alt.selection_single(fields = ["Year"], bind=input_dropdown, name = ' |')

#Checkbox
#input_checkbox = alt.binding_checkbox()
#selector = alt.selection_single(fields=["Year"], bind = input_checkbox, name = '1999')


color = alt.condition(selector,
                      alt.Color('Year:O', scale=alt.Scale(scheme='dark2'), legend=None),
                      alt.value('grey'))
opacity = alt.condition(selector,
                        alt.value(.85),
                        alt.value(.25))

time_chart = alt.Chart(Top_3_Filled_Capacity_Monthly_Average).mark_line(point = True).encode(
    x = 'Month:O',
    color = color,
    opacity = opacity,
    row = "Reservoir",
    tooltip = ["Reservoir", "Year", "Percent Filled", "Month"]
).add_selection(
    selector
).transform_filter(
    selector
).interactive()

legend  = alt.Chart(Top_3_Filled_Capacity_Monthly_Average).mark_point().encode(
    y = alt.Y('Year:O', axis = alt.Axis(orient = 'right')),
    color = color
).add_selection(
    selector
)

legend | (time_chart.encode(y='Storage:Q') | (time_chart.encode(y='Percent Filled:Q')| time_chart.encode(y='Delta Percent Filled:Q')))

alt.HConcatChart(...)